# Human Silhouette & Coordinate-based Tracking
- Sillhouette can be manually resized 
- To make sure silhouette fits within the screen, be sure to modify values<br> **move_x_by, move_y_by** too 
- Optical flow is given manual coordinates for hand

In [6]:
import cv2
import numpy as np

In [7]:
def show_img(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Get image and generate inverted mask of the image

In [8]:
import os
os.getcwd()

'c:\\cv\\opencv\\Scripts\\local_repo\\2017-2-CSP-MuJe-6\\Won-Joon'

### Load image & Create mask

In [9]:
# Read images
img1 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette.png")
# img2 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette (480 x 640).png") # not used!


# Frame is 640 x 480, resize img by 1/3
#resize_factor = 1.8
img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2gray = cv2.resize(img2gray, (int(480/1.8), int(640/1.4)))



# create mask from silhoutte
ret, mask = cv2.threshold(img2gray, 230, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# show_img(img2gray)
# show_img(mask_inv)

### Sparse Optical Flow settings

In [10]:
# ShiTomasi corner detection parameters
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Lucas Kanade optical flow parameters
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Random colors
color = np.random.randint(0,255,(100,3))

### Take first frame & assign features

In [117]:
def make_features_xy(x, x2, y, y2):
    return [ [list(item)] for item in zip(list(range(x, x2)), list(range(y, y2)))]

In [128]:
test_list = make_features_xy(100, 120, 100, 120)

In [129]:
test_list

[[[100, 100]],
 [[101, 101]],
 [[102, 102]],
 [[103, 103]],
 [[104, 104]],
 [[105, 105]],
 [[106, 106]],
 [[107, 107]],
 [[108, 108]],
 [[109, 109]],
 [[110, 110]],
 [[111, 111]],
 [[112, 112]],
 [[113, 113]],
 [[114, 114]],
 [[115, 115]],
 [[116, 116]],
 [[117, 117]],
 [[118, 118]],
 [[119, 119]]]

In [134]:
# Parameters for displaying silhouette 
y, x = img2gray.shape
move_y_by = 20
move_x_by = 200

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_frame_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = [[100, 100]]

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)


cap = cv2.VideoCapture(0)
while ( cap.isOpened() ):
    _, frame = cap.read()

    # Put silhouette on ROI
    roi = frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ]
    bg_roi_withMask = cv2.bitwise_and(roi, roi, mask = mask_inv)
    
    destination = cv2.add( frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] , bg_roi_withMask)
    frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] = destination

    
    

    cv2.imshow('feed', frame)
    
    ch = 0xFF & cv2.waitKey(1)
    if ch == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        

error: C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:10638: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [135]:
cap.release()

Spare code

In [ ]:
# Parameters for displaying silhouette 
y, x = img2gray.shape
move_y_by = 20
move_x_by = 200

# Take first frame and find corners in it
cap = cv2.VideoCapture(0)

ret, old_frame = cap.read()
old_frame_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = np.array(   test_list  , dtype = 'float32' )
print(p0.shape)
cap.release()

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)


cap = cv2.VideoCapture(0)
while ( cap.isOpened() ):
    _, frame = cap.read()    
    
    # Part 2:calculate optical flow ##############################
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevImg = old_gray,
                                           nextImg = frame_gray, 
                                           prevPts = p0, 
                                           nextPts = None, **lk_params)
#    print("p1:", p1)
    
    # Select good points    
    good_new = p1[st==1]
    good_old = p0[st==1]
 #   print(type(good_new))
 #   print("good_new: ", good_new, "\n\n")
    
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    
    
     # Part 1: Put silhouette on ROI ###########################
    roi = frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ]
    bg_roi_withMask = cv2.bitwise_and(roi, roi, mask = mask_inv)
    
    destination = cv2.add( frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] , bg_roi_withMask)
    frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] = destination


    
    
    cv2.imshow('feed', frame)
    ch = 0xFF & cv2.waitKey(1)
    if ch == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        